## Hands on session 2.4 - highly composite derivations

This session returns to the calculation tree derivations first shown in session 1.3, building on these tools and demonstrating a real use case for them.

The file `anisotropy.py` is supplied which contains a collection of functions used in more complex derivations.

Demonstrated concepts:

- Building functions out of node calculations
- Applying range filter cutoffs in derivations
- Additive derivations
- Implicit temperature derivations


In [1]:
import numpy as np

import holoviews as hv
import matplotlib.pyplot as plt
import panel as pn

import anisotropy as aniso
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm
import RMK_support.IO_support as io
from RMK_support import RKWrapper, Grid, treeDerivation, Node
import RMK_support.dashboard_support as ds

/Users/aca22/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'anisotropy'

### The system
A true model of an anisotropic plasma would require a full set of fluid equations. For the sake of brevity this is not done here. Nonetheless, one integral part of an anisotropic plasma model is the isotropisation of temperatures. It is this process that is demonstrated here in this notebook.

The equations shown here can be found in [Chodura et al.](link)

We will look at the simplified one species case (for electrons) first. In this case, the anisotropy is quantified by the variable 

$$
    X = \frac{T_\perp}{T_\parallel} - 1
$$

With these variables defined, the equations that govern the system are:
$$
    \frac{\partial W_\perp}{\partial t} = 2 n \nu \left[T_\perp (K_{002} - K_{200}) \right]
$$
$$
    \frac{\partial W_\parallel}{\partial t} = 4 n \nu \left[ T_\parallel (K_{200} - K_{002}) \right]
$$
where $\nu$ is in our case an arbitrary collisionality.
$K_{LMN}$ are results of taking moments of a bi-Maxwellian:
$$
    K_{200} = \frac{1}{X} \left[ -1 + (1 + X)\varphi (X) \right]
$$
$$
    K_{002} = \frac{2}{X} \left[ 1 - \varphi (X) \right]
$$
where $\varphi$ for $X>0$:
$$
    \varphi = \frac{\arctan{(\sqrt{X})}}{\sqrt{X}}
$$
and for $X<0$:
$$
    \varphi = \frac{\log{\frac{1+\sqrt{-X}}{1-\sqrt{-X}}}}{2\sqrt{-X}}
$$

### Basic setup

In a full model evolving all fluid variables we would include many spatial derivative terms. However, since we're only interested in the 0D effect of temperature isotropisation we can reduce the number of cells simulated as much as possible. Because we want to write the setup in a general way, using staggered grids, we need at least 2 cells.

In [ ]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_2_4/"
rk.setHDF5Path(hdf5Filepath) 

#Grid initialization
xGridWidths = np.ones(2)
rk.grid = Grid(xGridWidths, interpretXGridAsWidths=True)

### Normalization
Alongside the standard ReMKiT1D normalisation parameters that are employed, it is worthwhile specifying the electron mass.

In [ ]:
elMass = 9.10938e-31 # electron mass
amu = 1.6605390666e-27  # atomic mass unit

rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

### Handling particle species data
As covered in workshop session 2.3, some built in models require knowledge of the species they are being constructed for. This is the case for the `implicitTemperatures()` model that is used in this notebook. As such, let us declare an electron species that will contain all of the associated implicit variables.

In [ ]:
rk.addSpecies("e", speciesID=0, atomicA=elMass/amu, charge=-1.0, associatedVars=["ne", "Ge", "WePar", "WePerp"])

electronSpecies = rk.getSpecies("e")

### Base variables
With the species now declared, it is time to introduce the base variables of the system. These are the implicit variables upon which the models and their respective derived terms are built upon. Before adding the variables to the wrapper, we shall create some initial conditions that will be assigned to the different variables. 

**NOTE** In an isotropic system the internal energy of an ideal gas can be written as $W = \frac32 n T$ (in normalised units). But due to the anisotropic nature of this system we must separate the degrees of freedom, hence $W_\perp = n T_\perp$ and $W_\parallel = \frac12 n T_\parallel$.

In [ ]:
# Set conserved variables in container
TePar = 2*np.ones(2)
TePerp = 5*np.ones(2)

ne = np.ones(2)

WePar = ne*TePar/2
WePerp = ne*TePerp

WIth these initial conditions declared, we can create variables for density, flux and internal energy:

In [ ]:
rk.addVarAndDual("ne", ne, isCommunicated=True)
rk.addVarAndDual("Ge", primaryOnDualGrid=True, isCommunicated=True)  # Ge_dual is evolved, and Ge is derived
rk.addVarAndDual("WePar", WePar, isCommunicated=True)
rk.addVarAndDual("WePerp", WePerp, isCommunicated=True)

We would like to have the temperatures implicit, but they should be related to the corresponding energy densities. Hence we label them as stationary variables, which can then be 'evolved' through the `implicitTemperatures()` model.

In [ ]:
rk.addVarAndDual("TePar", TePar, isStationary=True, isCommunicated=True)
rk.addVarAndDual("TePerp", TePerp, isStationary=True, isCommunicated=True)

### Anisotropic variables
The anisotropic variables that are derived using the implicit variables declared above are: $\nu$, $K_{200}$ & $K_{002}$. We can now add these to the ReMKiT1D wrapper:

In [ ]:
rk.addVar("collFreq", np.ones(2),isDerived=True) # Collision frequency set arbitrarily

rk.addCustomDerivation("K200", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="200")))
rk.addCustomDerivation("K002", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="002")))

There is one issue with these terms, namely as the system approaches isotropy, the value of $X$ will aproach zero. This is problematic as the numerical solver will break due to a divide by zero error. Hence, the terms $K_{200}$ and $K_{002}$ need to have a form that is used only when X approaches zero. This situation is a prime candidate to make use of ReMKiT1D's `rangeFilterDerivation()` function. 

To begin, let's create the variable $X$ as well as derivations for $K_{200}$ $K_{002}$ when $X$ is small:

In [ ]:
rk.addVarAndDual("X",
                 isDerived=True,
                 derivationRule=sc.derivationRule("XDeriv", ("TePar","TePerp")),
                 derivOptions=treeDerivation(Node("TePerp")/Node("TePar") - 1)) # YOUR CODE HERE?

rk.addCustomDerivation("K200Small", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="200",smallX=True)))

rk.addCustomDerivation("K002Small", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="002",smallX=True)))

We can use the `rangeFilterDerivation()` function found in `simple_containers` to write what the values of $K_{200}$ and $K_{002}$ are in the different regimes. Since $-\infty < X < \infty$ we must assign three different ranges in which $K_{200}$ and $K_{002}$ must be calculated. These being:
$$
    \begin{cases}
        -1e16 < X < -0.01 \\
        |X| << 0.01 \\
        0.01 > X < 1e16 \\
    \end{cases}
$$
**Note** that the range is not actually $-\infty < X < \infty$, since the solver deals with floating point numbers we must give a large float as the range.

The `rangeFilterDerivation()` is passed a list of variables that are required to create the derivation. Firstly the derivation that will be filtered by range is given. The control variable that is used to specify the range is provided, this is done by passing the index of the control variable from the list of variables passed to the `rangeFilterDerivation()`. Next the range of values that the control variable may have is specified. The variables that are required by the derivation are specified, this is done by providing the index of the variables used in the derivation.

This is also explained in the docstring of the `rangeFilterDerivation()`:

In [ ]:
help(sc.rangeFilterDerivation)

We add a derivation for both $K_{200}$ and $K_{002}$ in all of the specified ranges. Making sure that for small values, the correct derivation is used.

In [ ]:
rk.addCustomDerivation("filterSmallValsK200", sc.rangeFilterDerivation("K200Small",controlIndices=[1],controlRanges=[[-0.01,0.01]],derivIndices=[2,3]))
rk.addCustomDerivation("filterPlusLargeK200", sc.rangeFilterDerivation("K200",controlIndices=[1],controlRanges=[[0.01,1e16]],derivIndices=[2,3]))
rk.addCustomDerivation("filterMinusLargeK200", sc.rangeFilterDerivation("K200",controlIndices=[1],controlRanges=[[-1e16,-0.01]],derivIndices=[2,3]))

rk.addCustomDerivation("filterSmallValsK002", sc.rangeFilterDerivation("K002Small",controlIndices=[1],controlRanges=[[-0.01,0.01]],derivIndices=[2,3]))
rk.addCustomDerivation("filterPlusLargeK002", sc.rangeFilterDerivation("K002",controlIndices=[1],controlRanges=[[0.01,1e16]],derivIndices=[2,3]))
rk.addCustomDerivation("filterMinusLargeK002", sc.rangeFilterDerivation("K002",controlIndices=[1],controlRanges=[[-1e16,-0.01]],derivIndices=[2,3]))

Since the ranges of the filtered functions do not overlap, we can simply sum them up and store them as a single variable so that $K_{200}$ and $K_{002}$ exist over the range $-1e16 < X < 1e16$. This can be done using the `additiveDerivation()` function found in `simple_containers`.

The `additiveDerivation()` requires a list of the derivations which are to be summed and stored as a single variable. The result can be raised to an arbitrary power. Similarly to the `rangeFilterDerivation()`, the `additiveDerivation()` is passed a list of variables that are used within the derivation. The indecies of the variables required for each derivation must be provided.

This is also explained in the docstring of the `additiveDerivation()`:

In [ ]:
help(sc.additiveDerivation)

We can now add the variables $K_{200}$ and $K_{002}$:

In [ ]:
rk.addVarAndDual("K200",
                 isDerived=True,
                 derivationRule=sc.derivationRule("filteredK200",["X","TePar","TePerp"]),
                 derivOptions=sc.additiveDerivation(["filterSmallValsK200","filterPlusLargeK200","filterMinusLargeK200"],resultPower=1., derivIndices=[[1,2,3]]*3))

rk.addVarAndDual("K002",
                 isDerived=True,
                 derivationRule=sc.derivationRule("filteredK002",["X","TePar","TePerp"]),
                 derivOptions=sc.additiveDerivation(["filterSmallValsK002","filterPlusLargeK002","filterMinusLargeK002"],resultPower=1., derivIndices=[[1,2,3]]*3))

### Implicit temperature derivation
Fluid models in ReMKiT1D are mostly written in conservative form. As such, the temperature is considered a derived variable. However, we can make use of the `implicitTemperatures()` function found in `common_models` to derive the temperature from the implicit fluid variables such that temperature is also an implicit variable.

In [ ]:
help(cm.implicitTemperatures)

In [ ]:
# Initializing model
implicitTempModelPar = cm.implicitTemperatures(modelTag="implicitTempElPar",
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePar"],
                                            speciesTemperatures=["TePar"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=1)

rk.addModel(implicitTempModelPar)

# Initializing model
implicitTempModelPerp = cm.implicitTemperatures(modelTag="implicitTempElPerp",
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePerp"],
                                            speciesTemperatures=["TePerp"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=2,
                                            ignoreKineticContribution=True)

rk.addModel(implicitTempModelPerp)

### Temperature Isotropisation
We have a small number of evolved variables, with many custom terms, all with diagonal stencils. This is a prime candidate for the use of the `addNodeMatrixTermModel()` function to nearly create the models in single lines.

In [ ]:
perpIsoEl = cm.addNodeMatrixTermModel(rk,
                                      modelTag="perpIsoEl", 
                                      evolvedVar="WePerp",
                                      termDefs=[(2*Node("collFreq")*Node("K002")/Node("TePerp"),"ne"),
                                                (-2*Node("collFreq")*Node("K200")/Node("TePerp"),"ne")]) # YOUR CODE HERE?

parIsoEl = cm.addNodeMatrixTermModel(rk,
                                      modelTag="parIsoEl", 
                                      evolvedVar="WePar",
                                      termDefs=[(4*Node("collFreq")*Node("TePerp")*Node("K200")/(Node("TePar")**2),"ne"),
                                                (-4*Node("collFreq")*Node("TePerp")*Node("K002")/(Node("TePar")**2),"ne")]) # YOUR CODE HERE?

### Time integration

In [ ]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['TePerp','TePar']) ) # we want all evolved variables to converge

rk.setIntegratorGlobalData(initialTimestep=0.1)

rk.addTermDiagnosisForVars(["WePar","WePerp"])

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(2500)
rk.setFixedStepOutput(50)

### Write config file

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
numFiles = 50
loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames,
                             isXinMeters=rk.grid.isLengthInMeters)
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
hv.output(size=100, dpi=150)

In [ ]:
pn.extension(comms="vscode")  # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData, rk.grid)

dashboard.fluid2Comparison().show()
